In [ ]:
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 30.4 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pulp as pl

In [ ]:
# Parameters
seat_capacity = 100
prob_showup = 0.94
revenue_per_seat = 400
voucher_cost = revenue_per_seat * 2
max_overbooking = 20

# Objective: Maximize the net revenue
# Decision variable: tickets_overbooked

# Initialize the Linear Program
lp = pl.LpProblem("Maximize_Net_Revenue", pl.LpMaximize)

# Decision variables: the number of overbooked tickets (from 0 to max_overbooking)
x = pl.LpVariable.dicts("Tickets_Overbooked", range(0, max_overbooking+1), lowBound=0, cat='Integer')

In [ ]:
# Objective Function: Maximize net revenue
def net_revenue(tickets_overbooked):
    tickets_sold = seat_capacity + tickets_overbooked
    total_showups = tickets_sold * prob_showup
    if total_showups <= seat_capacity:
        return revenue_per_seat * total_showups
    else:
        upset_customers = total_showups - seat_capacity
        return (total_showups * revenue_per_seat) - (voucher_cost * upset_customers)


In [ ]:
# Adding the objective to the problem
lp += pl.lpSum([net_revenue(t) * x[t] for t in range(0, max_overbooking+1)]), "Total_Net_Revenue"

# Constraints:
# The number of overbooked tickets should not exceed the maximum allowed.
lp += pl.lpSum([x[t] for t in range(0, max_overbooking+1)]) <= max_overbooking, "Max_Overbooking_Constraint"

# Solve the problem
lp.solve()



1

In [ ]:
# Print the optimal solution
print("Status:", pl.LpStatus[lp.status])

# Optimal number of overbooked tickets
optimal_overbooking = np.argmax([x[t].varValue for t in range(0, max_overbooking+1)])
print(f"Optimal Tickets Overbooked: {optimal_overbooking}")

# Print the optimal revenue
optimal_revenue = pl.value(lp.objective)
print(f"Optimal Net Revenue: ${optimal_revenue:.2f}")

Status: Optimal
Optimal Tickets Overbooked: 6
Optimal Net Revenue: $797120.00
